# Evaluate speaker verification with PCA

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.chdir('../..')
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

In [ ]:
import torch
from sklearn.decomposition import PCA
import numpy as np

from notebooks.notebooks_utils import (
    load_models,
    evaluate_models,
    create_metrics_df
)

from sslsv.evaluations.CosineSVEvaluation import CosineSVEvaluation, CosineSVEvaluationTaskConfig

In [ ]:
models = load_models([
    './models/old/vox2_ddp_sntxent_s=30_m=0/config.yml',
])

In [ ]:
class CosineSVWithPCAEvaluation(CosineSVEvaluation):
    
    def _prepare_evaluation(self):
        super()._prepare_evaluation()
        
        # Train PCA
        Z = np.array([t.mean(dim=0).numpy() for t in self.test_embeddings.values()])
        self.pca = PCA(n_components=200)
        self.pca.fit(Z)
    
    def _get_sv_score(self, a, b):
        enrol = self.test_embeddings[a]
        test = self.test_embeddings[b]
        
        enrol = torch.from_numpy(self.pca.transform(enrol.numpy()))
        test = torch.from_numpy(self.pca.transform(test.numpy()))

        score = self._compute_score(enrol, test)

        return score.item()

In [ ]:
evaluate_models(models, CosineSVEvaluation, CosineSVEvaluationTaskConfig())
create_metrics_df(models)

In [ ]:
evaluate_models(models, CosineSVWithPCAEvaluation, CosineSVEvaluationTaskConfig())
create_metrics_df(models)